In [1]:
%matplotlib inline

from consts import *
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
from keras.models import Model
from joblib import dump, load
import keras
import tensorflow as tf
import json
import glob
import shutil

USE_SVM = False

img_dir = base_path + 'images/negative_raw_test/'
ss_info_filepath = base_path + 'neg_ss_info.csv'

In [ ]:
# check whether we've already done selective search on images in directory
CHECK_PREV = True
if CHECK_PREV and os.path.isfile(ss_info_filepath):
    df_neg_ss = pd.read_csv(ss_info_filepath)
    paths_list = df_neg_ss['path'].tolist()
else:
    CHECK_PREV = False

ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss_file = open(ss_info_filepath, "a+")
    
for fpath in glob.glob(img_dir + '*.*'):    
    if CHECK_PREV:
        if fpath in paths_list:
            continue

    # run selective search
    image = cv2.imread(img_path)[...,::-1] # switch to rgb

    # initialise selective search
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    all_ssresults.append(ssresults)
    images.append(image)
    print(ssresults.shape)

    ss_dict = {'results': np.ndarray.tolist(ssresults)[:2000]}
    
    ss_file.write(fpath + ',' + '\"' + json.dumps(ss_dict) + '\"' + '\n')

ss_file.close()

In [2]:
# load up the CNN weights
weights_dir = base_path + 'weights/'

from CNN_model import VGG_model, custom_model

if USE_SVM:
    # model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
    custom_model.load_weights(weights_dir + 'cnn_0_120_good.h5')
    model = Model(inputs=model.input,
                       outputs=model.layers[-2].output)
else:
    VGG_model.load_weights(weights_dir + 'cnn_tuned_checkpoint.h5')
    model = VGG_model
model.summary()


Num GPUs Available:  1
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 30

In [30]:
import time
start_time = time.time()

def resize_norm(crop):
    resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
    if not USE_GREY:
        resized_norm = (resized/255) - 0.5
    else:
        resized_norm = resized - 0.5
    resized_norm = np.expand_dims(resized_norm, axis=0)
    return resized_norm

def limit_size(x, y, w, h):
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if x + w >= im_w:
        w = im_w - x - 1
    if y + h >= im_h:
        h = im_h - y - 1
    return (x, y, w, h)

true_neg_path = base_path + 'images/negative_true/'
false_neg_path = base_path + 'images/negative_false/'

if not os.path.exists(true_neg_path):
    os.mkdir(true_neg_path)
if not os.path.exists(false_neg_path):
    os.mkdir(false_neg_path)
    
log_file_path = base_path + 'rcnn_log.csv'
log_file = open(log_file_path, "a+")


ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

for fpath in glob.glob(img_dir + '*.*'):
    # run selective search
    image = cv2.imread(fpath)[...,::-1] # switch to rgb
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = image

    crop_list = []
    for i,result in enumerate(ssresults):
        if i >= 2000:
            break
        x,y,w,h = result

        if w < MIN_SIZE or h < MIN_SIZE or h > MAX_SIZE or w > MAX_SIZE:
            continue
        rect = Rectangle([x ,y], w, h, fill=False, 
                                color='green', lw=0.5)
    #         ax.add_patch(rect)

        crop = imout[y:y+h,x:x+w]
        resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)

        if not USE_GREY:
            resized_norm = (resized/255) - 0.5
        else:
            resized_norm = resized - 0.5

        resized_norm = np.expand_dims(resized_norm, axis=0)

        if USE_SVM:
            layer_output = np.array(model(resized_norm))
            prob = clf.predict_proba(layer_output)
        else:
            prob = model.predict(resized_norm)
        if prob[0][1] > 0.8:
            crop = [x, y, w, h]
            crop_list.append([crop, prob[0][1], resized])

    im_h, im_w, _ = image.shape 

    plt.figure(figsize=(18,18))
    ax = plt.gca()
    ax.imshow(image)
    sorted_crop_list = list(sorted(crop_list, key=lambda item:item[1], reverse=True))

    max_k = len(sorted_crop_list)
    if max_k > 3:
        max_k = 3

    k = 0
    begin_flag = 0
    prob_results = []
    for crop_img in sorted_crop_list:
        if k >= max_k:
            break
        crop_dims, prob, resized = crop_img
        # only consider the first entry < 0.5 probability
#         if prob < 0.5 and (k != 0 or begin_flag):
#             break
#         begin_flag = 1
        x, y, w, h = crop_dims
        rect = Rectangle([x ,y], w, h, fill=False, color='green', lw=1)
        ax.add_patch(rect)

        probs = []
        
        # test an image slightly smaller
        new_w = int(w*0.95)
        new_h = int(h*0.95)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])    
        
        # test an image even larger
        new_w = int(w*1.05)
        new_h = int(h*1.05)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])
        
        # test an image even larger
        new_w = int(w*1.1)
        new_h = int(h*1.1)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])
        
        # test an image even larger
        new_w = int(w*1.15)
        new_h = int(h*1.15)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])  

        # test an image even larger
        new_w = int(w*1.2)
        new_h = int(h*1.2)
        new_x = x - int((new_w - w)/2)
        new_y = y - int((new_h - h)/2)
        new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
        new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
        probs.append(model.predict(resize_norm(new_crop))[0][1])   

        min_prob = float(min(probs))
        max_prob = float(max(probs))
        test_prob = float(sum(probs) / len(probs))
        
        all_probs = [float(prob), min_prob, max_prob, test_prob]
        prob_results.append(all_probs)
        
        barrier1 = 0.90
        barrier2 = 0.985
        if w < 150 and h < 150:
            barrier1 = 0.99
            barrier2 = 0.99
        if min_prob < barrier1:
            continue
#             if min_prob < 0.2:
#                 continue
#             elif not (test_prob > 0.7 and max_prob > barrier2 and min_prob > 0.5):
#                 continue
#             else:
#                 col = 'yellow'
        else:
            col = 'red'
                
#         if prob < 0.5:
#             col = 'blue'
        prob_str = str(prob)[0:6]
        prob_str += ', ' + str(min_prob)[0:6]
        
        ax.text(x,y + 3, prob_str, c=col, ha='left', va='top', bbox=dict(facecolor='white', alpha=0.25, lw=0))
        rect = Rectangle([x ,y], w, h, fill=False, 
                            color=col, lw=1)
        ax.add_patch(rect)
        k += 1
        
    fname = fpath.split('/')[-1]
    if k == 0:
        img_dest = true_neg_path
    else:
        img_dest = false_neg_path 
        save_file = img_dest + fname.split('.')[0] + '(1)' + '.png'
        plt.savefig(save_file, bbox_inches='tight')
    shutil.move(fpath, img_dest + fname)
    plt.close()
    
    results_dict = {'results': prob_results}
    log_line = '{},{},{},\"{}\"\n'.format(fname.split('.')[0], len(sorted_crop_list), k, 
                                          json.dumps(results_dict))
    log_file.write(log_line)
log_file.close()

print(time.time() - start_time)

KeyboardInterrupt: 